In [ ]:
# Zelle 1: Importe und Konfiguration
import os
import pickle
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaLLM
import gradio as gr
import random

# Definieren Sie die Pfade
MAIN_SCRIPT_PATH = "./BS2zsm.pdf"
LITERATURE_DIR = "./Zusatz"
CACHE_DIR = "./cache"
VECTORSTORE_PATH = os.path.join(CACHE_DIR, "chroma_db")
METADATA_PATH = os.path.join(CACHE_DIR, "metadata.pkl")

# Stellen Sie sicher, dass das Cache-Verzeichnis existiert
os.makedirs(CACHE_DIR, exist_ok=True)

# Importiere die Module aus dem bs2_tutor Paket
from data_loader import load_pdfs, create_chunks, load_vectorstore, create_vectorstore
from tutor import OptimizedBS2Tutor
from gradio_interface import GradioInterface

In [ ]:
# Zelle 2: Ausführung der Hauptfunktion
def main():
    # Prüfe, ob der Vektorspeicher bereits existiert
    if os.path.exists(VECTORSTORE_PATH) and os.path.exists(METADATA_PATH):
        print("Lade existierenden Vektorspeicher...")
        vectorstore, metadata = load_vectorstore(VECTORSTORE_PATH, METADATA_PATH)
    else:
        print("Erstelle neuen Vektorspeicher...")
        # Lade PDFs
        documents = load_pdfs(MAIN_SCRIPT_PATH, LITERATURE_DIR)

        # Erstelle Chunks
        chunks = create_chunks(documents)

        # Erstelle Vektorspeicher
        vectorstore, metadata = create_vectorstore(chunks, VECTORSTORE_PATH, METADATA_PATH)

    # Initialisiere den Tutor
    tutor = OptimizedBS2Tutor(vectorstore)

    # Erstelle und starte das Gradio-Interface
    interface = GradioInterface(tutor)
    demo = interface.create_interface()
    return demo, tutor, vectorstore

# Führe die Hauptfunktion aus
demo, tutor, vectorstore = main()

In [ ]:
# Zelle 3: Starte das Interface
demo.launch(share=True)